In [ ]:
import os
import sys
import zipfile

import brayns
import phaneron
import bluepy
import seaborn as sns
import time
import math

from fpdf import FPDF

from luigi import Parameter

from bbp_workflow.circuit import BraynsService


mesh_folder = '/gpfs/bbp.cscs.ch/project/proj42/entities/dev/atlas/20181210-meshes'
mesh_names = ['SLM.stl', 'SO.stl', 'SP.stl', 'SR.stl']

clip_plane_definition_file = '/tmp/bbp-workflow/files/planes.txt'

image_resolution = (3840,2160)
image_samples_per_pixel = 16
image_areas_of_interest = 5
image_morphology_density = 1.0
image_somas_only = False
image_export_folder = '/var/lib/nginx/html/imgs'

circuit_model_index = 4


def prnt(txt):
    print(txt)
    sys.stdout.flush()


class MorphologyCollage(object):

    def __init__(self, url, circuit_configuration, clip_plane_definition_file):
        """
        :param brayns_url: URL to running instance of Brayns
        :param circuit_configuration: Configuration of the circuit
        :param clip_plane_definition_file: File containing the clipping plane definition
        """
        self._circuit_configuration = circuit_configuration
        self._clip_plane_definition_file = clip_plane_definition_file
        self._brayns = brayns.Client(url)
        self._circuit_explorer = phaneron.CircuitExplorer(self._brayns)
        self._clipping_planes = list()
        self._morphological_types = list()

    def load_morphological_types(self):
        """
        Loads morphological types from a given circuit
        """
        circuit = bluepy.Circuit(self._circuit_configuration)
        self._morphological_types = circuit.v2.cells.mtypes

    def remove_existing_models(self):
        """
        Remove existing 3D models from Brayns. Just to make sure we start the script in a clean state
        """
        prnt('Cleanup existing models...')
        model_ids = list()
        for model in self._brayns.scene.models:
            model_ids.append(model['id'])
        self._brayns.remove_model(model_ids)

    def load_meshes(self, folder, files):
        """
        Loads the meshes
        :param folder: Folder containing the meshes
        :param files: List of file names
        """
        prnt('Loading meshes...')
        for file in files:
            filename = folder + '/' + file
            prnt('- ' + filename)
            self._brayns.add_model(path=filename)

    def load_clipping_planes(self):
        """
        Loads the clipping planes definition from a text file
        """
        prnt('Loading clipping planes...')
        self._clipping_planes = list()
        with open(self._clip_plane_definition_file) as f:
            for line in f:
                values = line.split()
                for v in values:
                    coordinates = v.split(',')
                    c = list()
                    for k in range(len(coordinates)):
                        c.append(float(coordinates[k]))
                    self._clipping_planes.append(c)

    @staticmethod
    def _cross(a, b):
        """
        Compute the cross product of two 3D vectors
        :param a: Vector 1
        :param b: Vector 2
        :return: Cross product
        """
        c = [a[1]*b[2] - a[2]*b[1],
             a[2]*b[0] - a[0]*b[2],
             a[0]*b[1] - a[1]*b[0]]
        return c

    def set_camera_position(self, model_index, clipping_plane_index):
        """
        Sets the camera position according the clipping plane and the bounding box of the loaded morphologies
        :param model_index: Index of the model containing the morphologies
        :param clipping_plane_index: Index of the clipping plane
        """
        height = 0
        model_is_loaded = False
        while not model_is_loaded:
            # Forces model update
            self._brayns.get_model_properties(id=self._brayns.scene.models[model_index]['id'])

            # Get information about the model
            model = self._brayns.scene.models[model_index]
            aabb_min = model['bounds']['min']
            aabb_max = model['bounds']['max']

            p = self._clipping_planes[clipping_plane_index * 2]
            direction = [p[0], p[1], p[2]]
            left = [1, 0, 0]

            aabb_diag = [0, 0, 0]
            for k in range(3):
                aabb_diag[k] = aabb_max[k] - aabb_min[k]

            # Compute camera height according to model size
            height = math.sqrt(aabb_diag[0]*aabb_diag[0] + aabb_diag[1]*aabb_diag[1] + aabb_diag[2]*aabb_diag[2])
            if math.isinf(height):
                # Model is not yet committed to the 3D scene (still loading...)
                time.sleep(1)
            else:
                model_is_loaded = True

        # Camera properties
        params = self._brayns.OrthographicCameraParams()
        params.height = height
        params.enable_clipping_planes = False
        self._brayns.set_camera_params(params, response_timeout=2)

        # Camera position
        up = self._cross(direction, left)
        aabb_center = self._brayns.scene.models[model_index]['transformation']['rotation_center']
        origin=[0, 0, 0]
        for k in range(3):
            origin[k] = aabb_center[k] - 2000 * direction[k] + 100 * up[k]

        self._circuit_explorer.set_camera(origin=origin, direction=direction, up=up)

    def set_materials(self, model_id, palette, shading_mode, opacity=1, clipped=False):
        """
        Sets the materials for a given model id
        :param model_id: Id of the model
        :param palette: List of float RGB values
        :param shading_mode: None, diffuse, electron, etc.
        :param opacity: Opacity of the material
        :param clipped: Clipped by clipping planes defined at the scene level
        """
        opacities = list()
        shading_modes = list()
        diffuse_colors = list()
        specular_colors = list()
        material_ids = list()
        emissions = list()
        clips = list()

        nb_materials = len(palette)
        for i in range(nb_materials):
            material_ids.append(i)
            opacities.append(opacity)
            diffuse_colors.append(palette[i])
            specular_colors.append((1, 1, 1))
            shading_modes.append(shading_mode)
            clips.append(clipped)

        self._circuit_explorer.set_materials(
            model_ids=[model_id], material_ids=material_ids,
            opacities=opacities, shading_modes=shading_modes,
            diffuse_colors=diffuse_colors, specular_colors=specular_colors,
            emissions=emissions, clips=clips, specular_exponents=list(), reflection_indices=list(),
            refraction_indices=list(), simulation_data_casts=list(), glossinesses=list())

    def set_meshes_colors(self):
        """
        Sets colors on the meshes
        """
        prnt('Setting meshes colors...')
        palette = sns.color_palette('rainbow', 4)
        for i in range(0, 4):
            model_id = self._brayns.scene.models[i]['id']
            self._circuit_explorer.set_material_extra_attributes(model_id)
            self.set_materials(model_id, [palette[i]], opacity=0.8, clipped=True,
                               shading_mode=self._circuit_explorer.SHADING_MODE_NONE)

    def set_morphologies_colors(self, model_index):
        """
        Sets colors on morphologies
        :param model_index: Index of the model containing the morphologies
        """
        prnt('Setting morphologies colors')
        nb_materials = 65
        palette = list()
        for i in range(nb_materials):
            palette.append((0, 0, 0))
        model_id = self._brayns.scene.models[model_index]['id']
        self._circuit_explorer.set_material_extra_attributes(model_id)
        self.set_materials(model_id, palette, opacity=1, clipped=False,
                           shading_mode=self._circuit_explorer.SHADING_MODE_NONE)

    def load_scene(self, index, target, soma_only=True, density=1):
        """
        Loads clipping planes and morphologies for the specified index
        :param index: Index in the list of loaded clipping planes
        :param target: Target to load
        :param soma_only: Loads only somas if True, loads full morphologies otherwise
        :param density: Density of the circuit between 0 and 1 (0 is nothing, 1 is 100%)
        """
        if self._brayns.get_clip_planes() is not None:
            for plane in self._brayns.get_clip_planes():
                self._brayns.remove_clip_planes([plane['id']])

        nb_models = len(self._brayns.scene.models)
        if nb_models == 5:
            model_id = self._brayns.scene.models[nb_models-1]['id']
            self._brayns.remove_model([model_id])

        plane_1 = [0, 0, 0, 0]
        plane_2 = [0, 0, 0, 0]
        for k in range(4):
            plane_1[k] = self._clipping_planes[index*2][k]
            plane_2[k] = self._clipping_planes[index*2+1][k]

        self._brayns.add_clip_plane(plane_1)
        self._brayns.add_clip_plane(plane_2)

        radius_multiplier = 1
        if soma_only:
            radius_multiplier = 5

        # Load circuit
        self._circuit_explorer.load_circuit(
            areas_of_interest=image_areas_of_interest,
            cell_clipping=True, circuit_color_scheme=self._circuit_explorer.CIRCUIT_COLOR_SCHEME_NEURON_BY_MTYPE,
            path=self._circuit_configuration, targets=[target],
            density=density,
            load_axon=False, load_dendrite=not soma_only, load_apical_dendrite=not soma_only,
            radius_multiplier=radius_multiplier)

    def set_rendering_settings(self):
        """
        Sets rendering settings
        """
        prnt('Setting rendering attributes...')
        self._brayns.set_renderer(current='advanced_simulation', background_color=(1, 1, 1))
        params = self._brayns.AdvancedSimulationRendererParams()
        params.gi_distance = 100
        params.shadows = 0
        params.soft_shadows = 1
        params.pixel_alpha = 1
        self._brayns.set_renderer_params(params)

    def set_orthographic_camera(self):
        """
        Sets orthographic camera as the active one (Brayns supports multiple types of cameras)
        """
        # Orthographic camera settings
        prnt('Selecting orthographic camera...')
        self._brayns.set_camera(current='orthographic')


def generate_pdf(image_folder, pdf_file):
    """
    Generates a pdf file out of a list of PNG images in a given folder
    :param image_folder: Folder containing PNG images
    :param pdf_file: PDF file
    """
    import glob

    extensions = ['*.png']
    margin = 10

    image_paths = list()
    for ext in extensions:
        image_paths.extend(glob.glob(os.path.join(image_folder, ext)))
    image_paths.sort()

    pdf = FPDF(unit="pt", format=[image_resolution[0] + 2 * margin, image_resolution[1] + 2 * margin])
    pdf.set_compression(False)
    for image_path in image_paths:
        pdf.add_page()
        pdf.image(image_path, margin, margin)

    pdf.output(pdf_file, 'F')

def generate_zip(zip_file_path, dir_path):
    dir_path = os.path.abspath(dir_path)
    with zipfile.ZipFile(zip_file_path, "w", compression=zipfile.ZIP_DEFLATED) as out_file:
        for archive_dir_path, dir_names, file_names in os.walk(dir_path):
            for file_name in file_names:
                file_path = os.path.abspath(os.path.join(archive_dir_path, file_name))
                zip_file_name = file_path[len(dir_path) + 1:]
                out_file.write(file_path, zip_file_name)


class Run(BraynsService):
    circuit_config = Parameter()
    clip_plane_definition_file = Parameter()

    def code(self, brayns_url):
        os.makedirs(image_export_folder, exist_ok=True)

        collage = MorphologyCollage(brayns_url, self.circuit_config, self.clip_plane_definition_file)
        collage.load_morphological_types()
        collage.remove_existing_models()
        collage.load_clipping_planes()
        collage.load_meshes(mesh_folder, mesh_names)
        collage.set_meshes_colors()
        collage.set_rendering_settings()
        collage.set_orthographic_camera()

        return
        
        nb_images = int(len(collage._clipping_planes) / 2)

        for type_i, morphological_type in enumerate(collage._morphological_types):
            prnt('Rendering morphological type ' + morphological_type)
            os.makedirs(f'{image_export_folder}/{morphological_type}', exist_ok=True)
            for i in range(nb_images):
                try:
                    self.set_status_message(f'{type_i} {morphological_type}: {i}/{nb_images}')
                    self.set_progress_percentage(int(i/nb_images * 100))
                    prnt(f'Scene {i}...')
                    collage.load_scene(i, target=morphological_type, soma_only=image_somas_only,
                                        density=image_morphology_density)
                    prnt('- Setting orthographic camera...')
                    collage.set_camera_position(circuit_model_index, i)
                    prnt('- Setting materials...')
                    collage.set_morphologies_colors(circuit_model_index)
                    file_name = f'{i:05d}.png'
                    image_filename = f'{image_export_folder}/{morphological_type}/{file_name}'
                    prnt(f'- Rendering frame {i}/100 to {image_filename}...')
                    collage._brayns.image(
                        samples_per_pixel=image_samples_per_pixel,
                        size=image_resolution,
                        format='png').save(image_filename)

                except Exception as e:
                    prnt(e)

            # disable pdfs
            # pdf_filename = f'{image_export_folder}/{morphological_type}.pdf'
            # prnt('Generating PDF file... ' + pdf_filename)
            # generate_pdf(f'{image_export_folder}/{morphological_type}', pdf_filename)

        zip_filename = f'{image_export_folder}/../report.zip'
        prnt('Generating ZIP file... ' + zip_filename)
        generate_zip(zip_filename, image_export_folder)

        self.set_tracking_url(f'http://bbp-workflow-web-{self.user}.ocp.bbp.epfl.ch/report.zip')

In [ ]:
runner = Code('r1i7n13:5000')